In [90]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import seaborn as sns
import torch
import pandas as pd
import torchvision
from tqdm.notebook import tqdm
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import Normalize
import torchsummary
from torch.optim import lr_scheduler
from pytorch_lightning.callbacks import Callback
import torch.nn as nn
from tempfile import TemporaryDirectory
import time
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf")  # For export
sns.reset_orig()
sns.set()

# Tensorboard extension (for visualization purposes later)
%load_ext tensorboard
# Setting the seed

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Device: cpu


## Create a training dataset

In [175]:
img = next(iter(train_dataloader))[0].float()[2]
(img - mean[:, None, None]).mean()

tensor(-0.7735, dtype=torch.float64)

In [2]:
shots = [16769, 16773]#, 16534, 16769, 16773, 18057]
shot_df = pd.DataFrame([])
for shot in shots:
    df = pd.read_csv(f'/compass/Shared/Users/bogdanov/vyzkumny_ukol/LHmode-detection-shot{shot}.csv')
    shot_df = pd.concat([shot_df, df], axis=0)

df_mode = shot_df['mode'].copy()
df_mode[shot_df['mode']=='L-mode']=0
df_mode[shot_df['mode']=='H-mode']=1
df_mode[shot_df['mode']=='ELM']=2
shot_df['mode'] = df_mode
shot_df = shot_df.reset_index(drop=True) #each shot has its own indexing starting from 0 to 2232

# Precalculated mean and std for each color and each image 
#TODO: Do i have to calculate mean and std everytime I add new shot do database?
mean, std = pd.read_csv('normalization.csv', index_col=0).values

In [200]:
class ImageDataset(Dataset):
    def __init__(self, annotations, img_dir, mean, std):
        self.img_labels = annotations #pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.loc[idx, 'filename'])
        image = read_image(img_path)
        normalized_image = (image - self.mean[:, None, None])/self.std[:, None, None]
        label = self.img_labels.iloc[idx, 1]
        return normalized_image, label
    

    
dataset = ImageDataset(annotations=shot_df[['filename', 'mode']], img_dir='/compass/Shared/Users/bogdanov/vyzkumny_ukol',\
                        mean=mean, std=std)

### TODO: adjust dataset sizes

In [201]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset, batch_size=6, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=6, shuffle=True)
val_dataloader = DataLoader(dataset, batch_size=6, shuffle=False)

In [202]:
dataloaders = {'train':train_dataloader, 'val':test_dataloader} #TODO: add test loader in train function and consequenlty to this dict
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}

## Import pretrained model

In [203]:
pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1')

num_ftrs = pretrained_model.fc.in_features
# Here the size of each output sample is set to 3.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
pretrained_model.fc = torch.nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM


### I will try to freeze all layers except 4-th and fc layer

In [204]:
for name, para in pretrained_model.named_parameters():
    if "layer4" in name or "fc" in name:
        continue
    else:
        para.requires_grad = False

### Alternatively freeze all the weights excepts those of last fc layer

In [205]:
# for param in pretrained_model.parameters():
#     param.requires_grad = False
 
# # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = pretrained_model.fc.in_features
# pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM

In [206]:
pretrained_model = pretrained_model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(pretrained_model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

### Here I simply print the structure of model for input with size (3,500,640)

In [207]:
#torchsummary.summary(pretrained_model, (3,500,640))

In [208]:
# for name, para in pretrained_model.named_parameters():
#     print("="*40)
#     print(f"name: {name}")
#     print(para)

## Training
### Training function was copied from [tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#finetuning-the-convnet)

In [209]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')
    
        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs) #2D tensor with shape Batchsize*len(class_names)
                        _, preds = torch.max(outputs, 1) #preds = 1D array of indicies of maximum values in row. ([2,1,2,1,2]) - third feature is largest in first sample, second in second...
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0) #!!! Why is it multiplied by batchsize???
                    running_corrects += torch.sum(preds == labels.data) #How many correct answers
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [210]:
model = train_model(pretrained_model, criterion, optimizer,
                         exp_lr_scheduler, num_epochs=2)

Epoch 0/1
----------


RuntimeError: expected scalar type Double but found Float

In [32]:
input_sample, output_sample = next(iter(train_dataloader))

In [36]:
input_sample.dtype

torch.uint8

In [34]:
pretrained_model(input_sample.float())

tensor([[-0.5713, -0.0479,  0.7213],
        [-0.4954, -0.0453,  0.5941],
        [-0.6617, -0.2476,  1.1831],
        [-0.3468, -0.2114,  0.2598],
        [-0.4050, -0.3050,  0.6333],
        [-0.4093, -0.3420,  0.9406],
        [-0.3759, -0.0757,  0.7771],
        [-0.3825, -0.2447,  0.9320],
        [-0.4870,  0.0302,  0.7899],
        [-0.5933, -0.3860,  0.7538],
        [-0.2675, -0.2473,  0.5913],
        [-0.3886, -0.2959,  0.9656],
        [-0.4664, -0.1463,  0.5332],
        [-0.4605, -0.4333,  0.6088],
        [-0.7232,  0.2062,  0.8629],
        [-0.3489, -0.2791,  0.7161],
        [-0.5243, -0.3131,  0.8363],
        [-0.3101, -0.1967,  0.2661],
        [-0.2075, -0.1874,  0.2482],
        [-0.4529, -0.3715,  0.6189],
        [-0.4606,  0.0211,  0.8148],
        [-0.2277, -0.2277,  0.2110],
        [-0.3889, -0.2456,  0.7082],
        [-0.3115, -0.4024,  0.5357],
        [-0.2493, -0.2659,  0.4713],
        [-0.2865, -0.0605,  0.9108],
        [-0.3793, -0.3426,  0.3289],
 